In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

df = pd.read_excel("life_data.xlsx")
label_name = '주거환경만족도'
cols = ['의료시설 수', '대중교통만족도', '문화시설 수', '경제력', '주택가격 안정화지수', '1인당 녹지 면적', '상업시설 수', '전체녹지', '주거환경만족도']
df.isnull().sum() # 결측치 있는지 확인

#폰트설정
f = [f.name for f in fm.fontManager.ttflist]
print(f)
plt.rc('font', family='Malgun Gothic')

#이상치 확인 후 제거
df = pd.DataFrame(df)
df = df.iloc[1:,2:]# 실수 데이터 설정
def boxplot_vis(df):
    plt.figure(figsize=(30, 30))
    for col_idx in range(len(df.columns)):
        plt.subplot(6, 2, col_idx+1) # 6행 2열 서브플롯에 각 feature 박스플롯 시각화
        plt.boxplot(df[df.columns[col_idx]], flierprops=dict(markerfacecolor='r', marker='D'))  # flierprops: 빨간색 다이아몬드 모양으로 아웃라이어 시각화
        plt.title("Feature"  + df.columns[col_idx], fontsize=20) # 그래프 타이틀: feature name
    plt.show()
boxplot_vis(df)# boxplot_vis 함수 호출

def remove_outlier(input_data):
    q1 = input_data.quantile(0.25) # 제 1사분위수
    q3 = input_data.quantile(0.75) # 제 3사분위수
    iqr = q3 - q1 # IQR(Interquartile range) 계산
    minimum = q1 - (iqr * 1.5) # IQR 최솟값
    maximum = q3 + (iqr * 1.5) # IQR 최댓값
    # IQR 범위 내에 있는 데이터만 산출(IQR 범위 밖의 데이터는 이상치)
    df_removed_outlier = input_data[(minimum < input_data) & (input_data < maximum)]
    return df_removed_outlier
df_prep = remove_outlier(df)
df_prep['target'] = 0

df_prep.dropna(axis = 0, how = 'any', inplace = True)
print(f"이상치 포함된 데이터 비율: {round((len(df) - len(df_prep))*100/len(df), 2)}%")
df_prep.to_csv('df_prep.csv')#전처리를 마친 데이터 저장

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import validation_curve, learning_curve, KFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor()

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df_prep[cols], df_prep[label_name], test_size=0.3, random_state=42)

# Define the range of values for the parameter
param_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
cv = KFold(n_splits = 5,random_state= 0,shuffle = True)
# Calculate validation curve
train_scores, test_scores = validation_curve(
    rf_regressor, X_train, y_train, param_name='n_estimators', param_range=param_range, scoring='neg_mean_squared_error')
train_sizes, train_scores, test_scores = learning_curve(rf_regressor, X_train, y_train, scoring='neg_mean_squared_error', train_sizes=np.linspace(0.1, 1.0, 10))


# Calculate mean and standard deviation of train scores
train_mean = -np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
# Calculate mean and standard deviation of test scores
test_mean = -np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot validation curve
plt.figure(figsize=(10, 6))
plt.plot(param_range, train_mean, label='Training error', color='blue')
plt.plot(param_range, test_mean, label='Validation error', color='red')

# Draw the bands
plt.fill_between(param_range, train_mean - train_std, train_mean + train_std, color='blue', alpha=0.1)
plt.fill_between(param_range, test_mean - test_std, test_mean + test_std, color='red', alpha=0.1)

# Add labels and title
plt.xlabel('Number of Estimators')
plt.ylabel('Mean Squared Error')
plt.title('Validation Curve')
plt.legend(loc='best')
plt.xticks(param_range)
plt.show()

# Plot the learning curve
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, label='Training error', color='blue')
plt.plot(train_sizes, test_mean, label='Validation error', color='red')

# Draw the bands
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color='blue', alpha=0.1)
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color='red', alpha=0.1)

# Add labels and title
plt.xlabel('Training Set Size')
plt.ylabel('Mean Squared Error')
plt.title('Learning Curve')
plt.legend(loc='best')
plt.show()

RandomForestRegressor(n_estimators=6)
rf_regressor.fit(X_train, y_train)
# Make predictions on the test data
y_pred = rf_regressor.predict(X_test)

# Calculate mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
import matplotlib.pyplot as plt

# Create scatter plot
plt.scatter(y_test, y_pred, color='blue', alpha=0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red')  # Diagonal line for reference
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Actual vs Predicted Values')
plt.show()


In [ ]:


rf_regressor = RandomForestRegressor( n_estimators = 6, random_state=42)

# Create the learning curve

# Calculate the mean and standard deviation of the train scores
train_mean = -np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

# Calculate the mean and standard deviation of the test scores
test_mean = -np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

